In [2]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

Implemention regression using sequential API of keras

In [13]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

In [14]:
model.compile(
    loss=keras.losses.mean_squared_error,
    optimizer="sgd"
)

In [16]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 4s 10ms/step - loss: 0.3948 - val_loss: 0.3792
Epoch 2/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3786 - val_loss: 0.3797
Epoch 3/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3795 - val_loss: 0.3741
Epoch 4/20
363/363 [==============================] - 3s 10ms/step - loss: 0.3707 - val_loss: 0.3695
Epoch 5/20
363/363 [==============================] - 3s 8ms/step - loss: 0.3767 - val_loss: 0.3679
Epoch 6/20
363/363 [==============================] - 3s 7ms/step - loss: 0.3671 - val_loss: 0.3722
Epoch 7/20
363/363 [==============================] - 3s 8ms/step - loss: 0.3667 - val_loss: 0.3700
Epoch 8/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3667 - val_loss: 0.3694
Epoch 9/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3624 - val_loss: 0.3651
Epoch 10/20
363/363 [==============================] - 3s 9ms/step - loss: 0.3752 - val_loss: 0.36

In [17]:
model_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 1s 7ms/step - loss: 0.3839


In [18]:
X_new = X_test[:3]
y_pred = model.predict(X_new)

In [22]:
y_pred.round(3)

array([[1.221],
       [0.825],
       [2.884]], dtype=float32)

In [20]:
y_test[:3]

array([1.139, 0.736, 2.1  ])

**Functional** API using keras

In [23]:
input_= keras.layers.Input(shape = X_train.shape[1:]) 
hidden1 = keras.layers.Dense(30, activation ="relu")(input_) 
hidden2 = keras.layers.Dense(30, activation ="relu")(hidden1) 
concat = keras.layers.Concatenate()([input_, hidden2]) 
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

compile and fit will be same as above

**Multiinput** API's keras

But what if you want to send a subset of the features through the wide path and a different subset (possibly overlapping) through the deep path.
Example, suppose we have 5 fearues(0 to 4) through wide path and six features through(2 to 7) deep path

In [30]:
input_A = keras.layers.Input(shape=[5], name ="wide_input") 
input_B = keras.layers.Input(shape=[6], name ="deep_input") 
hidden1 = keras.layers.Dense(30, activation ="relu")(input_B) 
hidden2 = keras.layers.Dense(30, activation ="relu")(hidden1) 
concat = keras.layers.concatenate([input_A, hidden2]) 
output = keras.layers.Dense(1, name ="output")(concat)

model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [32]:
model.compile(loss=keras.losses.mean_squared_error, 
             optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [34]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:] 
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:] 
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[: 3], X_test_B[: 3]

In [35]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20, validation_data=((X_valid_A, X_valid_B), y_valid))

Epoch 1/20
363/363 [==============================] - 5s 13ms/step - loss: 2.5954 - val_loss: 0.9113
Epoch 2/20
363/363 [==============================] - 4s 10ms/step - loss: 0.7651 - val_loss: 0.6602
Epoch 3/20
363/363 [==============================] - 4s 10ms/step - loss: 0.6467 - val_loss: 0.6166
Epoch 4/20
363/363 [==============================] - 4s 10ms/step - loss: 0.6132 - val_loss: 0.5951
Epoch 5/20
363/363 [==============================] - 3s 9ms/step - loss: 0.5940 - val_loss: 0.5804
Epoch 6/20
363/363 [==============================] - 3s 8ms/step - loss: 0.5799 - val_loss: 0.5712
Epoch 7/20
363/363 [==============================] - 3s 9ms/step - loss: 0.5676 - val_loss: 0.5604
Epoch 8/20
363/363 [==============================] - 3s 8ms/step - loss: 0.5588 - val_loss: 0.5533
Epoch 9/20
363/363 [==============================] - 3s 8ms/step - loss: 0.5496 - val_loss: 0.5458
Epoch 10/20
363/363 [==============================] - 3s 9ms/step - loss: 0.5429 - val_loss: 0.

In [36]:
model.evaluate((X_test_A, X_test_B), y_test)

162/162 [==============================] - 1s 6ms/step - loss: 0.5168


0.516836404800415

In [37]:
y_pred = model.predict((X_new_A, X_new_B))

In [38]:
y_pred

array([[1.2890582],
       [1.6821191],
       [2.5410814]], dtype=float32)

In [39]:
y_test[:3]

array([1.139, 0.736, 2.1  ])

To make **multioutput** API's in keras

In [40]:
input_A = keras.layers.Input(shape=[5], name ="wide_input") 
input_B = keras.layers.Input(shape=[6], name ="deep_input") 
hidden1 = keras.layers.Dense(30, activation ="relu")(input_B) 
hidden2 = keras.layers.Dense(30, activation ="relu")(hidden1) 
concat = keras.layers.concatenate([input_A, hidden2]) 
output = keras.layers.Dense(1, name ="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)

model = keras.Model(inputs=[input_A, input_B], 
                   outputs=[output, aux_output])

In [41]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

**Observations**
here we need to specify loss for each of the output and weights are the importants given to each of the loss

In [42]:
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=20, 
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))

Epoch 1/20
363/363 [==============================] - 5s 12ms/step - loss: 1.0016 - main_output_loss: 0.9087 - aux_output_loss: 1.8374 - val_loss: 0.6514 - val_main_output_loss: 0.5883 - val_aux_output_loss: 1.2197
Epoch 2/20
363/363 [==============================] - 4s 11ms/step - loss: 0.6029 - main_output_loss: 0.5441 - aux_output_loss: 1.1329 - val_loss: 0.5556 - val_main_output_loss: 0.5027 - val_aux_output_loss: 1.0322
Epoch 3/20
363/363 [==============================] - 4s 11ms/step - loss: 0.5316 - main_output_loss: 0.4864 - aux_output_loss: 0.9384 - val_loss: 0.5083 - val_main_output_loss: 0.4677 - val_aux_output_loss: 0.8738
Epoch 4/20
363/363 [==============================] - 4s 11ms/step - loss: 0.5644 - main_output_loss: 0.5334 - aux_output_loss: 0.8436 - val_loss: 0.4928 - val_main_output_loss: 0.4598 - val_aux_output_loss: 0.7894
Epoch 5/20
363/363 [==============================] - 4s 10ms/step - loss: 0.4997 - main_output_loss: 0.4714 - aux_output_loss: 0.7549 - val

In [43]:
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))

162/162 [==============================] - 1s 6ms/step - loss: 0.4055 - main_output_loss: 0.3893 - aux_output_loss: 0.5512


In [44]:
total_loss, main_loss, aux_loss

(0.40551453828811646, 0.38933122158050537, 0.5511640906333923)

In [45]:
y_pred_main, y_pred_aux = model.predict((X_test_A, X_test_B))

**Subclassing API's**

In [46]:
class WideAndDeepModel(keras.Model): 
    def __init__(self, units=30, activation="relu", **kwargs): 
        super().__init__(**kwargs) # handles standard args (e.g., name) 
        self.hidden1 = keras.layers.Dense(units, activation=activation) 
        self.hidden2 = keras.layers.Dense(units, activation=activation) 
        self.main_output = keras.layers.Dense(1) 
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B) 
        hidden2 = self.hidden2(hidden1) 
        concat = keras.layers.concatenate([input_A, hidden2]) 
        main_output = self.main_output(concat) 
        aux_output = self.aux_output(hidden2) 
        return main_output, aux_output

In [47]:
model = WideAndDeepModel()

Saving the Model

In [49]:
# model = keras.models.Sequential([...]) # or keras.Model([...]) 
# model.compile([...]) 
# model.fit([...]) 
# model.save("my_keras_model.h5")

Loading the model

In [51]:
# model = keras.models.load_model("my_keras_model.h5")

**Callbacks**

using callbacks to save model during training

In [52]:
# [...] # build and compile the model 
# checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5") 
# history = model.fit( X_train, y_train, epochs = 10, callbacks =[checkpoint_cb])

for saving best model

Only saves best model out of the all the epochs. the following code is easy way to implement early stopping.

In [54]:
# checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True) 
# history = model.fit(X_train, y_train, epochs = 10, validation_data=(X_valid, y_valid), 
#                     callbacks =[checkpoint_cb]) 
# model = keras.models.load_model("my_keras_model.h5") # roll back to best model

Another way to implement early stoping is to use EarlyStopping callbacks. With passing the patience argument

In [55]:
# early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True) 
# history = model.fit(X_train, y_train, epochs=100, validation_data =(X_valid, y_valid), 
#                     callbacks =[early_stopping_cb])

We can specify both early stopping and checkpoints

In [56]:
# history = model.fit(X_train, y_train, epochs=100, validation_data =(X_valid, y_valid), 
#                     callbacks =[checkpoints_cb, early_stopping_cb])

**Custom Callback**

Code that prints ratio between validation loss and train loss after each epoch

In [57]:
class PrintValTrainRatioCallback(keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs): 
        print("\nval/train: {:.2f}".format(logs["val_loss"]/logs["loss"]))

As you might expect, you can implement on_train_begin(), on_train_end(), on_epoch_begin(), on_epoch_end(), on_batch_begin(), 
and on_batch_end(). 

Callbacks can also be used during evaluation and predictions, should you ever need them(e.g.,for debugging). 
For evaluation, you should implement on_test_begin(), on_test_end(), on_test_batch_begin(), or on_test_batch_end() (called by evaluate()).For prediction you should implement on_predict_begin(), on_predict_end(), on_predict_batch_begin(), or on_predict_batch_end() (called by predict()).

**Tensorboard for Visulizations**

In [59]:
import os 
root_logdir = os.path.join(os.curdir, "my_logs") 

def get_run_logdir(): 
    import time 
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S") 
    return os.path.join(root_logdir, run_id) 

run_logdir = get_run_logdir() # e.g., './my_logs/run_2019_06_07-15_15_22'

In [60]:
# Build and compile your model 
# tensorboard_cb = keras.callbacks.TensorBoard(run_logdir) 
# history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[tensorboard_cb])

**Hyperparameter Tuning**

In [73]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    model.compile(loss=keras.losses.mean_squared_error, 
                 optimizer=keras.optimizers.SGD(learning_rate=learning_rate))
    return model

In [74]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

C:\Users\hp\AppData\Local\Temp/ipykernel_26684/1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


Now we can use fit(), transform() and score() methos as normal sci-kit learn

In [75]:
keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 5s 8ms/step - loss: 1.3842 - val_loss: 0.6909
Epoch 2/100
363/363 [==============================] - 2s 6ms/step - loss: 0.6443 - val_loss: 0.6017
Epoch 3/100
363/363 [==============================] - 2s 6ms/step - loss: 0.5728 - val_loss: 0.5505
Epoch 4/100
363/363 [==============================] - 2s 5ms/step - loss: 0.5327 - val_loss: 0.5196
Epoch 5/100
363/363 [==============================] - 2s 6ms/step - loss: 0.5073 - val_loss: 0.5007
Epoch 6/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4941 - val_loss: 0.4866
Epoch 7/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4810 - val_loss: 0.4775
Epoch 8/100
363/363 [==============================] - 2s 6ms/step - loss: 0.4723 - val_loss: 0.4758
Epoch 9/100
363/363 [==============================] - 2s 6ms/step - loss: 0.4668 - val_loss: 0.4648
Epoch 10/100
363/363 [==============================] - 2s 6ms/step - loss: 0.4607 - val_lo

Note that fit() method will pass the extra arguments to other underlying keras model

In [78]:
keras_reg.score(X_test, y_test)

162/162 [==============================] - 1s 4ms/step - loss: 0.3776


-0.37761637568473816

In [77]:
y_pred = keras_reg.predict(X_new)

Applying Randomized Search which is good when we have lots of hyperparameters 

In [79]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal

In [80]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3], 
    "n_neurons": np.arange(1, 100), 
    "learning_rate": reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3) 
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), 
                 callbacks=(keras.callbacks.EarlyStopping(patience=10)))

Epoch 1/100
242/242 [==============================] - 4s 13ms/step - loss: 1.5198 - val_loss: 0.7523
Epoch 2/100
242/242 [==============================] - 3s 11ms/step - loss: 0.7103 - val_loss: 0.6778
Epoch 3/100
242/242 [==============================] - 3s 11ms/step - loss: 0.6443 - val_loss: 0.6339
Epoch 4/100
242/242 [==============================] - 3s 11ms/step - loss: 0.6003 - val_loss: 0.5970
Epoch 5/100
242/242 [==============================] - 3s 11ms/step - loss: 0.5677 - val_loss: 0.5659
Epoch 6/100
242/242 [==============================] - 2s 9ms/step - loss: 0.5404 - val_loss: 0.5415
Epoch 7/100
242/242 [==============================] - 2s 9ms/step - loss: 0.5183 - val_loss: 0.5205
Epoch 8/100
242/242 [==============================] - 3s 11ms/step - loss: 0.4996 - val_loss: 0.5037
Epoch 9/100
242/242 [==============================] - 4s 16ms/step - loss: 0.4858 - val_loss: 0.4894
Epoch 10/100
242/242 [==============================] - 2s 8ms/step - loss: 0.4734 -

242/242 [==============================] - 2s 7ms/step - loss: 0.3678 - val_loss: 0.3729
Epoch 61/100
242/242 [==============================] - 2s 8ms/step - loss: 0.3681 - val_loss: 0.3724
Epoch 62/100
242/242 [==============================] - 2s 8ms/step - loss: 0.3653 - val_loss: 0.3720
Epoch 63/100
242/242 [==============================] - 2s 9ms/step - loss: 0.3648 - val_loss: 0.3696
Epoch 64/100
242/242 [==============================] - 2s 8ms/step - loss: 0.3647 - val_loss: 0.3733
Epoch 65/100
242/242 [==============================] - 2s 8ms/step - loss: 0.3639 - val_loss: 0.3691
Epoch 66/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3631 - val_loss: 0.3677
Epoch 67/100
242/242 [==============================] - 2s 8ms/step - loss: 0.3615 - val_loss: 0.3659
Epoch 68/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3606 - val_loss: 0.3694
Epoch 69/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3587 - val_loss: 

Epoch 40/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3720 - val_loss: 0.3831
Epoch 41/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3707 - val_loss: 0.3832
Epoch 42/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3699 - val_loss: 0.3860
Epoch 43/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3687 - val_loss: 0.3796
Epoch 44/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3674 - val_loss: 0.3805
Epoch 45/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3664 - val_loss: 0.3789
Epoch 46/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3661 - val_loss: 0.3772
Epoch 47/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3633 - val_loss: 0.3772
Epoch 48/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3621 - val_loss: 0.3766
Epoch 49/100
242/242 [==============================] - 2s 7ms/step - loss: 0.3616

Epoch 11/100
242/242 [==============================] - 2s 8ms/step - loss: 0.6305 - val_loss: 0.6185
Epoch 12/100
242/242 [==============================] - 2s 8ms/step - loss: 0.6200 - val_loss: 0.6090
Epoch 13/100
242/242 [==============================] - 2s 8ms/step - loss: 0.6105 - val_loss: 0.6006
Epoch 14/100
242/242 [==============================] - 2s 8ms/step - loss: 0.6023 - val_loss: 0.5936
Epoch 15/100
242/242 [==============================] - 2s 8ms/step - loss: 0.5948 - val_loss: 0.5865
Epoch 16/100
242/242 [==============================] - 2s 9ms/step - loss: 0.5883 - val_loss: 0.5808
Epoch 17/100
242/242 [==============================] - 3s 11ms/step - loss: 0.5823 - val_loss: 0.5752
Epoch 18/100
242/242 [==============================] - 2s 8ms/step - loss: 0.5771 - val_loss: 0.5703
Epoch 19/100
242/242 [==============================] - 2s 9ms/step - loss: 0.5723 - val_loss: 0.5657
Epoch 20/100
242/242 [==============================] - 2s 9ms/step - loss: 0.568

242/242 [==============================] - 2s 7ms/step - loss: 0.4937 - val_loss: 0.5056
Epoch 41/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4912 - val_loss: 0.5028
Epoch 42/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4889 - val_loss: 0.5005
Epoch 43/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4865 - val_loss: 0.4981
Epoch 44/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4843 - val_loss: 0.4964
Epoch 45/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4822 - val_loss: 0.4942
Epoch 46/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4802 - val_loss: 0.4920
Epoch 47/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4782 - val_loss: 0.4903
Epoch 48/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4763 - val_loss: 0.4883
Epoch 49/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4746 - val_loss: 

...

Epoch 64/100
242/242 [==============================] - 2s 8ms/step - loss: 0.4511 - val_loss: 0.4507
Epoch 65/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4503 - val_loss: 0.4500
Epoch 66/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4496 - val_loss: 0.4496
Epoch 67/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4488 - val_loss: 0.4491
Epoch 68/100
242/242 [==============================] - 2s 7ms/step - loss: 0.4481 - val_loss: 0.4486
Epoch 69/100
242/242 [==============================] - 1s 6ms/step - loss: 0.4472 - val_loss: 0.4474
Epoch 70/100
242/242 [==============================] - 1s 6ms/step - loss: 0.4466 - val_loss: 0.4468
Epoch 71/100
242/242 [==============================] - 2s 6ms/step - loss: 0.4459 - val_loss: 0.4465
Epoch 72/100
242/242 [==============================] - 2s 6ms/step - loss: 0.4452 - val_loss: 0.4452
Epoch 73/100
242/242 [==============================] - 2s 6ms/step - loss: 0.4446

Epoch 23/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5306 - val_loss: 0.5367
Epoch 24/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5297 - val_loss: 0.5360
Epoch 25/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5288 - val_loss: 0.5354
Epoch 26/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5280 - val_loss: 0.5349
Epoch 27/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5273 - val_loss: 0.5344
Epoch 28/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5266 - val_loss: 0.5338
Epoch 29/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5259 - val_loss: 0.5333
Epoch 30/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5253 - val_loss: 0.5328
Epoch 31/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5248 - val_loss: 0.5323
Epoch 32/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5242

242/242 [==============================] - 1s 5ms/step - loss: 0.5429 - val_loss: 0.5406
Epoch 83/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5424 - val_loss: 0.5401
Epoch 84/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5419 - val_loss: 0.5398
Epoch 85/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5415 - val_loss: 0.5395
Epoch 86/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5411 - val_loss: 0.5389
Epoch 87/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5407 - val_loss: 0.5384
Epoch 88/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5403 - val_loss: 0.5379
Epoch 89/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5399 - val_loss: 0.5375
Epoch 90/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5395 - val_loss: 0.5372
Epoch 91/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5392 - val_loss: 

Epoch 62/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5331 - val_loss: 0.5351
Epoch 63/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5327 - val_loss: 0.5347
Epoch 64/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5323 - val_loss: 0.5343
Epoch 65/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5319 - val_loss: 0.5343
Epoch 66/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5314 - val_loss: 0.5336
Epoch 67/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5312 - val_loss: 0.5334
Epoch 68/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5308 - val_loss: 0.5331
Epoch 69/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5305 - val_loss: 0.5327
Epoch 70/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5301 - val_loss: 0.5328
Epoch 71/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5298

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x00000206265B6FA0>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000205468609D0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [82]:
rnd_search_cv.best_params_

{'learning_rate': 0.0018999855145031622, 'n_hidden': 2, 'n_neurons': 83}